In [2]:
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import MonthEnd, QuarterEnd, QuarterBegin
from dateutil import parser
import time
#!pip install seaborn
import seaborn as sns

from numpy.random import randint

In [ ]:
conn = wrds.Connection()

1a

In [2]:
df1=pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\bond_yields_pt1.csv')

In [3]:
df1_1=pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\bond_yields_pt2.csv')

In [4]:
df2= df1.append(df1_1) #combined data as excel couldn't handle all the data at once

In [5]:
df2['trd_exctn_dt']= pd.to_datetime(df2['trd_exctn_dt'],format='%Y-%m-%d') #change date to datetime dtype

In [6]:
df2 = df2[df2['yld_pt'].notna()] #drop rows with no yields
df2 = df2.drop(columns=['bond_sym_id', 'company_symbol']) #drop bond symbol and company symbol columns.
df2 = df2.sort_values(by=['CUSIP_ID', 'trd_exctn_dt', 'trd_exctn_tm', 'yld_pt'], ascending=True) #sorted by cusip, date,time and yield
df2_ordered = df2
df2_ordered

,CUSIP_ID,yld_pt,trd_exctn_dt,trd_exctn_tm
891065,00077TAB0,6.525827,2015-01-05,14:27:45
891064,00077TAB0,6.608047,2015-01-05,14:27:45
891067,00077TAB0,6.786439,2015-01-06,10:23:34
891066,00077TAB0,6.842175,2015-01-06,10:23:34
891069,00077TAB0,6.897267,2015-02-17,11:24:15
...,...,...,...,...
3417,98934KAB6,2.971758,2017-11-30,17:23:00
3419,98934KAB6,3.009462,2017-12-04,09:27:55
3418,98934KAB6,3.210928,2017-12-04,09:27:55
3420,98934KAB6,2.977512,2017-12-19,12:22:54


In [7]:
#Dropped duplicate dates and kept last occurence
df3 = df2_ordered.drop_duplicates(subset=['CUSIP_ID', 'trd_exctn_dt'], keep='last')

In [8]:
#grouped by bond and each month contained sorted yields
Grouped = df3.groupby(['CUSIP_ID',
    pd.Grouper(key=('trd_exctn_dt'),freq='M')])
Grouped = Grouped.apply(lambda _df3: _df3.sort_values(by=['yld_pt'],ascending=True))

In [9]:
Grouped

CUSIP_ID    yld_pt trd_exctn_dt trd_exctn_tm
CUSIP_ID  trd_exctn_dt                                                      
00077TAB0 2015-01-31   891064  00077TAB0  6.608047   2015-01-05     14:27:45
                       891066  00077TAB0  6.842175   2015-01-06     10:23:34
          2015-02-28   891068  00077TAB0  7.002821   2015-02-17     11:24:15
          2015-03-31   101170  00077TAB0  6.816467   2015-03-31     14:59:18
          2015-04-30   101172  00077TAB0  6.029157   2015-04-24     12:10:55
...                                  ...       ...          ...          ...
98934KAB6 2017-11-30   3402    98934KAB6  2.936286   2017-11-22     09:27:49
                       3417    98934KAB6  2.971758   2017-11-30     17:23:00
                       3400    98934KAB6  3.036031   2017-11-14     15:54:28
          2017-12-31   3421    98934KAB6  2.960962   2017-12-19     12:22:55
                       3418    98934KAB6  3.210928   2017-12-04     09:27:55

[113786 rows x 4 columns]

In [10]:
Grouped = Grouped.reset_index(drop=True)

In [11]:
gp = Grouped

In [12]:
#ranked each yield by bond and sorted yield for each month
gp['rank'] = gp.groupby(['CUSIP_ID',
    pd.Grouper(key=('trd_exctn_dt'), freq='m')])['yld_pt'].rank()

In [51]:
gp

,CUSIP_ID,yld_pt,trd_exctn_dt,trd_exctn_tm,rank
0,00077TAB0,6.608047,2015-01-05,14:27:45,1.0
1,00077TAB0,6.842175,2015-01-06,10:23:34,2.0
2,00077TAB0,7.002821,2015-02-17,11:24:15,1.0
3,00077TAB0,6.816467,2015-03-31,14:59:18,1.0
4,00077TAB0,6.029157,2015-04-24,12:10:55,1.0
...,...,...,...,...,...
113781,98934KAB6,2.936286,2017-11-22,09:27:49,2.0
113782,98934KAB6,2.971758,2017-11-30,17:23:00,3.0
113783,98934KAB6,3.036031,2017-11-14,15:54:28,4.0
113784,98934KAB6,2.960962,2017-12-19,12:22:55,1.0


In [13]:
gp2 = gp.groupby(['CUSIP_ID',
    pd.Grouper(key=('trd_exctn_dt'), freq='m')])

In [44]:
gp2

In [14]:
#find median of the ranks for each group
med=gp2['rank'].transform('median') 

In [31]:
med

0         1.5
1         1.5
2         1.0
3         1.0
4         1.0
         ... 
113781    2.5
113782    2.5
113783    2.5
113784    1.5
113785    1.5
Name: rank, Length: 113786, dtype: float64

In [15]:
#rounds the median to whole number 
med2 = np.ceil(med) 

In [32]:
med2

0         2.0
1         2.0
2         1.0
3         1.0
4         1.0
         ... 
113781    3.0
113782    3.0
113783    3.0
113784    2.0
113785    2.0
Name: rank, Length: 113786, dtype: float64

In [16]:
#selects median observation and removes ranked column
final_1a = gp[gp['rank']==med2].drop('rank',axis=1)

In [17]:
#Removed trade execution time
final_1a= final_1a.drop(columns='trd_exctn_tm')

In [18]:
final_1a

,CUSIP_ID,yld_pt,trd_exctn_dt
1,00077TAB0,6.842175,2015-01-06
2,00077TAB0,7.002821,2015-02-17
3,00077TAB0,6.816467,2015-03-31
4,00077TAB0,6.029157,2015-04-24
6,00077TAB0,6.492385,2015-06-10
...,...,...,...
113764,98934KAB6,2.807131,2017-08-09
113771,98934KAB6,2.858460,2017-09-26
113777,98934KAB6,2.955087,2017-10-24
113782,98934KAB6,2.971758,2017-11-30


1a complete

1b 

Downloaded data from CRSP.dsf


Used PERMCO to find the stock returns 

In [3]:
dfret=pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\stock_return.csv')
dfret

,PERMNO,date,RET
0,14593,2014-01-02,-0.014064
1,14593,2014-01-03,-0.021966
2,14593,2014-01-06,0.005453
3,14593,2014-01-07,-0.007156
4,14593,2014-01-08,0.006338
...,...,...,...
155592,16851,2017-12-22,0.006561
155593,16851,2017-12-26,-0.009291
155594,16851,2017-12-27,0
155595,16851,2017-12-28,0.00098


In [4]:
#Changed data type into appriopriate types
dfret['date']= pd.to_datetime(dfret['date'])
dfret['RET'] = pd.to_numeric(dfret['RET'], errors='coerce')

In [5]:
dfret1 = dfret.sort_values(by=['PERMNO', 'date'],) #sorted by PERMNO
dfret1

,PERMNO,date,RET
108207,10145,2014-01-02,-0.011492
108208,10145,2014-01-03,0.002214
108209,10145,2014-01-06,-0.000994
108210,10145,2014-01-07,0.004092
108211,10145,2014-01-08,-0.003965
...,...,...,...
146987,93013,2017-12-22,0.001886
146988,93013,2017-12-26,-0.001345
146989,93013,2017-12-27,0.006642
146990,93013,2017-12-28,-0.007936


In [6]:
dfstdev = dfret1.groupby('PERMNO')[['RET']].rolling(180).std() #calculated 180 day stdev
dfstdev

RET
PERMNO                 
10145  108207       NaN
       108208       NaN
       108209       NaN
       108210       NaN
       108211       NaN
...                 ...
93013  146987  0.010455
       146988  0.010445
       146989  0.010446
       146990  0.010463
       146991  0.010431

[155597 rows x 1 columns]

In [7]:
dfstdev1= dfstdev.rename(columns={'RET':'st_dev'}) #renamed to st dev

In [8]:
dfstdev1

st_dev
PERMNO                 
10145  108207       NaN
       108208       NaN
       108209       NaN
       108210       NaN
       108211       NaN
...                 ...
93013  146987  0.010455
       146988  0.010445
       146989  0.010446
       146990  0.010463
       146991  0.010431

[155597 rows x 1 columns]

In [9]:
dfstdev1 = dfstdev1.reset_index(level=0)

In [10]:
left = dfret1

In [11]:
right = dfstdev1

In [17]:
#combined data
data = pd.concat([dfret1, dfstdev1], axis=1,)
data

,PERMNO,date,RET,PERMNO,st_dev
108207,10145,2014-01-02,-0.011492,10145,NaN
108208,10145,2014-01-03,0.002214,10145,NaN
108209,10145,2014-01-06,-0.000994,10145,NaN
108210,10145,2014-01-07,0.004092,10145,NaN
108211,10145,2014-01-08,-0.003965,10145,NaN
...,...,...,...,...,...
146987,93013,2017-12-22,0.001886,93013,0.010455
146988,93013,2017-12-26,-0.001345,93013,0.010445
146989,93013,2017-12-27,0.006642,93013,0.010446
146990,93013,2017-12-28,-0.007936,93013,0.010463


In [15]:
data.columns = ['PERMNO', 'date', 'RET', 'PERMNO2','st_dev']

In [16]:
data

,PERMNO,date,RET,PERMNO2,st_dev
108207,10145,2014-01-02,-0.011492,10145,NaN
108208,10145,2014-01-03,0.002214,10145,NaN
108209,10145,2014-01-06,-0.000994,10145,NaN
108210,10145,2014-01-07,0.004092,10145,NaN
108211,10145,2014-01-08,-0.003965,10145,NaN
...,...,...,...,...,...
146987,93013,2017-12-22,0.001886,93013,0.010455
146988,93013,2017-12-26,-0.001345,93013,0.010445
146989,93013,2017-12-27,0.006642,93013,0.010446
146990,93013,2017-12-28,-0.007936,93013,0.010463


In [ ]:
data1 = data.drop(columns='PERMNO2')

In [13]:
data1

,PERMNO,date,RET,st_dev
108207,10145,2014-01-02,-0.011492,NaN
108208,10145,2014-01-03,0.002214,NaN
108209,10145,2014-01-06,-0.000994,NaN
108210,10145,2014-01-07,0.004092,NaN
108211,10145,2014-01-08,-0.003965,NaN
...,...,...,...,...
146987,93013,2017-12-22,0.001886,0.010455
146988,93013,2017-12-26,-0.001345,0.010445
146989,93013,2017-12-27,0.006642,0.010446
146990,93013,2017-12-28,-0.007936,0.010463


In [13]:
#filtered for selected dates
final1b = data1[(data1['date'] > '2015-01-01') & (data1['date'] < '2017-12-31')]
final1b

,PERMNO,date,RET,st_dev
108459,10145,2015-01-02,0.003103,0.010046
108460,10145,2015-01-05,-0.019056,0.010150
108461,10145,2015-01-06,-0.002339,0.010152
108462,10145,2015-01-07,0.007238,0.010164
108463,10145,2015-01-08,0.018219,0.010237
...,...,...,...,...
146987,93013,2017-12-22,0.001886,0.010455
146988,93013,2017-12-26,-0.001345,0.010445
146989,93013,2017-12-27,0.006642,0.010446
146990,93013,2017-12-28,-0.007936,0.010463
